In [1]:
# xyz connection test

import pyvisa as visa

rm = visa.ResourceManager()
print(rm.list_resources())

('ASRL3::INSTR',)


In [1]:
# Initialize program

from ni9215_interface import NI9215
import pyvisa as visa
from xyz_functions_gui import Home, Move_XYZ, Servo_off, Servo_on
import time
import matplotlib.pyplot as plt

In [2]:
# Test parameters (adjustable)

centerX = 101070
centerY = 69000
centerZ = 58500
displaceZ = 1000
port = 'COM3'

cycles = 2

In [11]:
# XYZ set up (brings it home)

rm = visa.ResourceManager()
print(rm.list_resources())
TTA = rm.open_resource(port)
Servo_on(TTA)
Home(TTA)
time.sleep(5)
print("Machine and DAQ set up.")

('ASRL3::INSTR',)


VisaIOError: VI_ERROR_SYSTEM_ERROR (-1073807360): Unknown system error (miscellaneous error).

In [10]:
# Position machine (centerX, centerY, centerZ)

Move_XYZ(TTA, 1, 1, 100, centerX, centerY, centerZ, 2)
print("Hammer positioned at surface.")

VisaIOError: VI_ERROR_SYSTEM_ERROR (-1073807360): Unknown system error (miscellaneous error).

In [ ]:
# Pressing cycle

for i in range(cycles):
    Move_XYZ(TTA, 1, 1, 30, centerX, centerY, centerZ + displaceZ, 10)
    Move_XYZ(TTA, 1, 1, 30, centerX, centerY, centerZ, 10)

In [9]:
# Close down machine

Home(TTA)
print("Returned home.")
time.sleep(5)
Servo_off(TTA)
print("Experiment complete! Plotting...")

Returned home.
Experiment complete! Plotting...


In [ ]:
# Initialize measurement

daq = NI9215(samplingRate=1000, numSamples=30000)
daq.startMeasure()

In [ ]:
# End measurement 

daq.endMeasure()

In [ ]:
# Plot

timeData = daq.data_index()
forceHammer = daq.data_a0()
voltagePZT = daq.data_a1()

fig, axs = plt.subplots(2, 1, constrained_layout=True)
axs[0].plot(timeData, forceHammer)
axs[0].set_title('Force Hammer Output Voltage')
axs[0].set_xlabel('time [non-dimensional index]')
axs[0].set_ylabel('Voltage [V]')
fig.suptitle('PZT and Force Hammer Setup Characterization', fontsize=13)

axs[1].plot(timeData, voltagePZT)
axs[1].set_xlabel('time (s)')
axs[1].set_title('PZT Output Voltage')
axs[1].set_ylabel('Voltage [V]')

plt.show()

In [7]:
# Go home

Home(TTA)